### Importing modules

In [1]:
import hoomd
import hoomd.md
import gsd, gsd.hoomd
import plato.draw.vispy as draw
import ipywidgets as widgets
import random
import numpy as np
import matplotlib.pyplot as pp

### Simulation setup

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.9.2 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 06/26/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, J Glaser, and S C Glotzer. "HOOMD-blue: A Python package for
  high-performance molecular dynamics and hard particle Monte Carlo
  simulations", Computational Materials Science 173 (2020) 109363
-----
HOOMD-blue is running on the CPU


In [3]:
# setting parameters

# number of particles, cube root
# represents number of particles along one side of our 3D box
num_particles_cubert = 10

# lattice constant defines spacing between particles
lattice_const = 3.0

# temperature settings
startTemp = 1.0
endTemp = 0.1

# set simulation length
timeSteps = 2e6

In [4]:
system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sc(a=lattice_const), n=num_particles_cubert)

# randomize velocities based on temperature
for p in system.particles:
    mass = p.mass;
    vx = random.gauss(0, startTemp / mass)
    vy = random.gauss(0, startTemp / mass)
    vz = random.gauss(0, startTemp / mass)
    p.velocity = (vx, vy, vz)

notice(2): Group "all" created containing 1000 particles


In [5]:
# generate the pair interaction potential
def LJG(r, rmin, rmax, r0, eps, sig2, Vscale):
    import math
    V = pow(r, -12) -  2 * pow(r, -6) - eps * math.exp(-pow(r - r0, 2) / (2 * sig2))
    F = 12 * pow(r, -13) - 12 * pow(r, -7) - eps * math.exp(-pow(r - r0, 2) / (2 * sig2)) * (r - r0) / sig2
    return (V / Vscale, F / Vscale)

# determine potential scaling
def determine_Vmin(rmax, r0, eps, sig2):
    Vmin = 0.0
    testNum = 10000
    for iTest in range(1, testNum):
        rTest = rmax / testNum * iTest
        VTest = LJG(rTest, 0.0, rmax, r0, eps, sig2, 1.0)[0]
        if (VTest < Vmin):
            Vmin = VTest
    return Vmin

# nl = hoomd.md.nlist.cell()
# lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl)
# lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0)
nl = hoomd.md.nlist.cell()
table = hoomd.md.pair.table(width = 1000, nlist = nl)
Vmin = determine_Vmin(2.5, 2.0, 4.5, 0.02)

table.pair_coeff.set('A', 'A', func = LJG, rmin = 0.5, rmax = 2.5,
                    coeff = dict(r0 = 2.0, eps = 4.5, sig2 = 0.02, Vscale = -Vmin))

In [6]:
# set integration parameters

hoomd.md.integrate.mode_standard(dt=0.005)

all = hoomd.group.all()
hoomd.md.integrate.nvt(group=all, 
                       kT=hoomd.variant.linear_interp(points = [(0, startTemp), (timeSteps, endTemp)]), 
                       tau=1.0)

In [7]:
# record data

# for visualization
hoomd.dump.gsd("dump1.gsd", period=timeSteps*1e-2, group=all, overwrite=True)

# for plots
hoomd.analyze.log(filename="dump.log",
                  quantities=['time', 'potential_energy', 'temperature'],
                  period=timeSteps*1e-2,
                  overwrite=True)

In [ ]:
# running simulation

hoomd.run(timeSteps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 1000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 21994 / 2000000 | TPS 2199.36 | ETA 00:14:59
Time 00:00:20 | Step 31023 / 2000000 | TPS 902.823 | ETA 00:36:20
Time 00:00:30 | Step 38889 / 2000000 | TPS 786.534 | ETA 00:41:33
Time 00:00:40 | Step 46515 / 2000000 | TPS 762.549 | ETA 00:42:41
Time 00:00:50 | Step 53866 / 2000000 | TPS 735.022 | ETA 00:44:07
Time 00:01:00 | Step 60872 / 2000000 | TPS 700.582 | ETA 00:46:07
Time 00:01:10 | Step 67981 / 2000000 | TPS 710.824 | ETA 00:45:17
Time 00:01:20 | Step 75073 / 2000000 | TPS 709.145 | ETA 00:45:14
Time 00:01:30 | Step 82219 / 2000000 | TPS 714.585 | ETA 00:44:43
Time 00:01:40 | Step 89349 / 2000000 | TPS 712.956 | ETA 00:44:39
Time 00:01:50 | Step 96443 / 2000000 | TPS 709.358 | ETA 00:44:43
Time 00:02:00 | Step 103604 / 20

Time 00:20:10 | Step 621267 / 2000000 | TPS 441.768 | ETA 00:52:00
Time 00:20:20 | Step 625662 / 2000000 | TPS 439.497 | ETA 00:52:07
Time 00:20:30 | Step 630082 / 2000000 | TPS 441.968 | ETA 00:51:39
Time 00:20:40 | Step 634471 / 2000000 | TPS 438.896 | ETA 00:51:51
Time 00:20:50 | Step 638867 / 2000000 | TPS 439.542 | ETA 00:51:36
Time 00:21:00 | Step 643269 / 2000000 | TPS 440.161 | ETA 00:51:22
Time 00:21:10 | Step 647618 / 2000000 | TPS 434.874 | ETA 00:51:49
Time 00:21:20 | Step 652009 / 2000000 | TPS 439.079 | ETA 00:51:10
Time 00:21:30 | Step 656369 / 2000000 | TPS 435.981 | ETA 00:51:21
Time 00:21:40 | Step 660737 / 2000000 | TPS 436.742 | ETA 00:51:06
Time 00:21:50 | Step 665098 / 2000000 | TPS 436.087 | ETA 00:51:01
Time 00:22:00 | Step 669478 / 2000000 | TPS 437.989 | ETA 00:50:37
Time 00:22:10 | Step 673839 / 2000000 | TPS 436.096 | ETA 00:50:40
Time 00:22:20 | Step 678231 / 2000000 | TPS 439.162 | ETA 00:50:09
Time 00:22:30 | Step 682611 / 2000000 | TPS 437.968 | ETA 00:5

Time 00:40:30 | Step 1145420 / 2000000 | TPS 427.073 | ETA 00:33:21
Time 00:40:40 | Step 1149716 / 2000000 | TPS 429.546 | ETA 00:32:59
Time 00:40:50 | Step 1153969 / 2000000 | TPS 425.269 | ETA 00:33:09
Time 00:41:00 | Step 1158215 / 2000000 | TPS 424.308 | ETA 00:33:03
Time 00:41:10 | Step 1162506 / 2000000 | TPS 429.02 | ETA 00:32:32
Time 00:41:20 | Step 1166783 / 2000000 | TPS 427.619 | ETA 00:32:28
Time 00:41:30 | Step 1171066 / 2000000 | TPS 428.247 | ETA 00:32:15
Time 00:41:40 | Step 1175357 / 2000000 | TPS 429.056 | ETA 00:32:01
Time 00:41:50 | Step 1179664 / 2000000 | TPS 430.622 | ETA 00:31:45
Time 00:42:00 | Step 1183929 / 2000000 | TPS 426.454 | ETA 00:31:53
Time 00:42:10 | Step 1188208 / 2000000 | TPS 427.848 | ETA 00:31:37
Time 00:42:20 | Step 1192483 / 2000000 | TPS 427.246 | ETA 00:31:30
Time 00:42:30 | Step 1196783 / 2000000 | TPS 429.748 | ETA 00:31:09
Time 00:42:40 | Step 1201083 / 2000000 | TPS 429.902 | ETA 00:30:58
Time 00:42:50 | Step 1205380 / 2000000 | TPS 429.

Time 01:00:40 | Step 1669281 / 2000000 | TPS 439.464 | ETA 00:12:32
Time 01:00:50 | Step 1673697 / 2000000 | TPS 441.573 | ETA 00:12:18
Time 01:01:00 | Step 1678094 / 2000000 | TPS 439.662 | ETA 00:12:12
Time 01:01:10 | Step 1682509 / 2000000 | TPS 441.45 | ETA 00:11:59
Time 01:01:20 | Step 1686927 / 2000000 | TPS 441.743 | ETA 00:11:48
Time 01:01:30 | Step 1691351 / 2000000 | TPS 442.346 | ETA 00:11:37
Time 01:01:40 | Step 1695768 / 2000000 | TPS 441.696 | ETA 00:11:28
Time 01:01:50 | Step 1700172 / 2000000 | TPS 440.359 | ETA 00:11:20
Time 01:02:00 | Step 1704590 / 2000000 | TPS 441.762 | ETA 00:11:08
Time 01:02:10 | Step 1708987 / 2000000 | TPS 439.632 | ETA 00:11:01
Time 01:02:20 | Step 1713426 / 2000000 | TPS 443.61 | ETA 00:10:46
Time 01:02:30 | Step 1717890 / 2000000 | TPS 446.332 | ETA 00:10:32
Time 01:02:40 | Step 1722318 / 2000000 | TPS 442.762 | ETA 00:10:27
Time 01:02:50 | Step 1726772 / 2000000 | TPS 445.393 | ETA 00:10:13
Time 01:03:00 | Step 1731205 / 2000000 | TPS 443.2

### Visualizing simulation

In [ ]:
def getFrameCount(fname):
    """
    inputs: fname, the filename (ex: 'dump.gsd')
    outputs: len(traj), number of frames in simulation
    """
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

In [ ]:
filename = 'dump.gsd'
frame_num = getFrameCount(filename)

In [ ]:
# function for centering droplet
def centercrystal(simulationbox,particlepositions):
    """
    Recentering the positions of all particles relative to COM, because otherwise they are split across the "box"
    Algorithm projects points along 3 orthogonal cylinders, finds COM, projects back into real space
    Useful resource w/ similar algorithm: https://www.cs.drexel.edu/~david/Papers/Bai_JGT.pdf
    
    inputs:
    simulationbox= 6x1 array of the box dimensions [Lx Ly Lz xy xz yz]
    particlepositions= the positions of the particles from the snapshot, #particles x 3 dimensions (xyz) array
    
    """    
    #center particles
    simbox=[simulationbox[3]-simulationbox[0], simulationbox[4]-simulationbox[1], simulationbox[5]-simulationbox[2]]
    theta = (particlepositions / simbox + .5) * 2 * np.pi
    sums = np.sum(np.exp(1j * theta), axis = 0)
    fractions = np.angle(sums) / 2 / np.pi
    fractions %= 1.
    fractions -= .5
    delta = fractions * simbox
    pos_CM=np.copy(particlepositions)
    pos_CM[:] -= delta[np.newaxis, :]
    
    # wrap particles back into box
    pos_CM[pos_CM[:, 0] > simulationbox[0]/2] -= [simulationbox[0], 0, 0]
    pos_CM[pos_CM[:, 1] > simulationbox[1]/2] -= [0, simulationbox[1], 0]
    pos_CM[pos_CM[:, 2] > simulationbox[2]/2] -= [0, 0, simulationbox[2]]
    pos_CM[pos_CM[:, 0] < -simulationbox[0]/2] += [simulationbox[0], 0, 0]
    pos_CM[pos_CM[:, 1] < -simulationbox[1]/2] += [0, simulationbox[1], 0]
    pos_CM[pos_CM[:, 2] < -simulationbox[2]/2] += [0, 0, simulationbox[2]]
    
    return pos_CM

In [ ]:
prim = draw.Spheres()
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2) #box
scene = draw.Scene([prim, box_prim], zoom=.5, clip_scale=5) #box
#scene = draw.Scene(prim) # no box
scene.show()

# looping over frames
@widgets.interact(frame_index=(0, frame_num-1, 1))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)  
        
        # grabbing particle positions and diameters from simulation file
        prim.positions = centercrystal(box, frame.particles.position)
        #prim.positions = frame.particles.position
        prim.diameters = np.full(len(frame.particles.position), 1)
        
        # setting particle colors
        colors = np.ones((len(prim.positions), 4))
        colors[:, :3] = np.float32(np.divide([255, 0, 0], 255)) #pink
        prim.colors = colors
    scene.render()

### Plotting simulation data over time

In [ ]:
data = np.genfromtxt(fname='dump.log', skip_header=True);

In [ ]:
pp.figure(figsize=(4,2.2), dpi=140)
pp.plot(data[:,1], data[:,2])
pp.xlabel('time')
pp.ylabel('potential energy')

In [ ]:
pp.figure(figsize=(4,2.2), dpi=140);
pp.plot(data[:,1], data[:,3]);
pp.xlabel('time');
pp.ylabel('temperature');